# iQuHACK 2026 - Unitaries Overview

This notebook loads and displays all 11 unitaries from the Clifford+T compilation challenge.

In [68]:
# Import Required Libraries
import numpy as np
from scipy.linalg import polar
import os
import sys

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, process_fidelity

# Import the optimizer (includes brute force analyzer functions)
from optimize_unitary import (
    load_unitary, decompose_and_optimize, apply_manual_identities, optimize_circuit,
    optimize_diagonal_chunks_with_rmsynth,
    # Brute force analyzer functions
    analyze_unitary, brute_force_search, quick_search, compare_circuits, full_analysis
)

# For nice matrix display
np.set_printoptions(precision=4, suppress=True, linewidth=120)

ImportError: cannot import name 'analyze_unitary' from 'optimize_unitary' (/Users/valentine/github/hackathons/iQuHack-2026/optimize_unitary.py)

## Helper Function

Load unitary matrices from `.npy` or `.txt` files and ensure unitarity via polar decomposition.

In [ ]:
def optimize_and_display(name, filepath, effort=3, verbose=False):
    """Load unitary, optimize it, and display results.
    
    Args:
        name: Display name for the unitary
        filepath: Path to the .npy file
        effort: Optimization effort level (1-3)
        verbose: If True, show detailed synthesis info (Z8 coefficients, etc.)
    """
    # Load the unitary
    U_target = load_unitary(filepath)
    n_qubits = int(np.log2(U_target.shape[0]))
    
    print(f"{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    # Redirect stdout to capture verbose output if not verbose
    if not verbose:
        import io
        from contextlib import redirect_stdout
        f = io.StringIO()
        with redirect_stdout(f):
            qc = decompose_and_optimize(U_target, effort)
            qc = apply_manual_identities(qc, epsilon=1e-10)
            qc = optimize_circuit(qc)
            qc = optimize_diagonal_chunks_with_rmsynth(qc, effort=effort, decoder="auto")
            qc = optimize_circuit(qc)
    else:
        qc = decompose_and_optimize(U_target, effort)
        qc = apply_manual_identities(qc, epsilon=1e-10)
        qc = optimize_circuit(qc)
        qc = optimize_diagonal_chunks_with_rmsynth(qc, effort=effort, decoder="auto")
        qc = optimize_circuit(qc)
    
    # Phase alignment
    U_actual = Operator(qc).data
    phase_diff = np.angle(np.trace(np.conj(U_target.T) @ U_actual))
    qc.global_phase = -phase_diff
    
    # Compute metrics
    U_final = Operator(qc).data
    dist = np.linalg.norm(U_target - U_final, ord=2)
    fid = process_fidelity(Operator(U_target), Operator(qc))
    
    # Display results
    ops = qc.count_ops()
    t_count = ops.get('t', 0) + ops.get('tdg', 0)
    
    print(f"\nTOTAL T-GATES: {t_count}")
    print(f"Gate counts: {dict(ops)}")
    print(f"Operator Norm Distance: {dist:.6e}")
    print(f"Process Fidelity: {fid:.6f}")
    
    # Print the circuit
    print(f"\nCircuit:")
    print(qc.draw(output='text', fold=120))
    print()
    
    return qc, U_target

---
## Unitary 1

In [ ]:
# Unitary 1
qc1, U1 = optimize_and_display("Unitary 1", "unitary1.npy", verbose=False)

Unitary 1

TOTAL T-GATES: 0
Gate counts: {'sdg': 1, 'cx': 1, 's': 1}
Operator Norm Distance: 0.000000e+00
Process Fidelity: 1.000000

Circuit:
     ┌─────┐┌───┐┌───┐
q_0: ┤ Sdg ├┤ X ├┤ S ├
     └─────┘└─┬─┘└───┘
q_1: ─────────■───────
                      



---
## Unitary 2

In [ ]:
# Unitary 2
qc2, U2 = optimize_and_display("Unitary 2", "unitary2.npy", verbose=False)

Unitary 2

TOTAL T-GATES: 0
Gate counts: {'cx': 2}
Operator Norm Distance: 2.239290e-01
Process Fidelity: 0.975085

Circuit:
     ┌───┐┌───┐
q_0: ┤ X ├┤ X ├
     └─┬─┘└─┬─┘
q_1: ──■────■──
               



---
## Unitary 3

In [ ]:
# Unitary 3
qc3, U3 = optimize_and_display("Unitary 3", "unitary3.npy", verbose=False)

Unitary 3

TOTAL T-GATES: 1
Gate counts: {'cx': 2, 'tdg': 1}
Operator Norm Distance: 5.609251e-02
Process Fidelity: 0.996856

Circuit:
global phase: π/8
                      
q_0: ──■───────────■──
     ┌─┴─┐┌─────┐┌─┴─┐
q_1: ┤ X ├┤ Tdg ├┤ X ├
     └───┘└─────┘└───┘



---
## Unitary 4

In [69]:
# Unitary 4
qc4, U4 = optimize_and_display("Unitary 4", "unitary4.npy", verbose=False)

Unitary 4

TOTAL T-GATES: 2
Gate counts: {'s': 7, 'h': 6, 'sdg': 3, 'cx': 2, 't': 2}
Operator Norm Distance: 1.121409e-01
Process Fidelity: 0.993722

Circuit:
global phase: 7π/4
     ┌───┐          ┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐          
q_0: ┤ S ├──────────┤ X ├┤ Sdg ├┤ H ├┤ T ├┤ H ├┤ S ├┤ X ├┤ Sdg ├──────────
     ├───┤┌───┐┌───┐└─┬─┘├─────┤├───┤├───┤├───┤├───┤└─┬─┘└┬───┬┘┌───┐┌───┐
q_1: ┤ S ├┤ S ├┤ H ├──■──┤ Sdg ├┤ H ├┤ T ├┤ H ├┤ S ├──■───┤ H ├─┤ S ├┤ S ├
     └───┘└───┘└───┘     └─────┘└───┘└───┘└───┘└───┘      └───┘ └───┘└───┘



---
## Unitary 5

In [ ]:
# Unitary 5
qc5, U5 = optimize_and_display("Unitary 5", "unitary5.npy", verbose=False)

Unitary 5

TOTAL T-GATES: 0
Gate counts: {'h': 4, 'cx': 3}
Operator Norm Distance: 4.454693e-16
Process Fidelity: 1.000000

Circuit:
global phase: π/4
          ┌───┐     ┌───┐     
q_0: ──■──┤ H ├──■──┤ H ├──■──
     ┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐
q_1: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘



---
## Unitary 6

In [ ]:
# Unitary 6
qc6, U6 = optimize_and_display("Unitary 6", "unitary6.npy", verbose=False)

Unitary 6

TOTAL T-GATES: 2
Gate counts: {'h': 4, 'tdg': 2, 'cx': 2}
Operator Norm Distance: 4.450419e-01
Process Fidelity: 0.822930

Circuit:
global phase: π/4
     ┌─────┐┌───┐          ┌───┐
q_0: ┤ Tdg ├┤ H ├──■────■──┤ H ├
     ├─────┤├───┤┌─┴─┐┌─┴─┐├───┤
q_1: ┤ Tdg ├┤ H ├┤ X ├┤ X ├┤ H ├
     └─────┘└───┘└───┘└───┘└───┘



---
## Unitary 7

In [ ]:
# Unitary 7
qc7, U7 = optimize_and_display("Unitary 7", "unitary7.npy", verbose=False)

Unitary 7

TOTAL T-GATES: 2078
Gate counts: {'h': 2095, 't': 2063, 's': 968, 'sdg': 17, 'tdg': 15, 'cx': 3}
Operator Norm Distance: 2.509845e-10
Process Fidelity: 1.000000

Circuit:
global phase: 0.43143
     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_0: ┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├»
     ├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤├───┤»
q_1: ┤ S ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_0: ┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ 

---
## Unitary 8

In [ ]:
# Unitary 8
qc8, U8 = optimize_and_display("Unitary 8", "unitary8.npy", verbose=False)

Unitary 8

TOTAL T-GATES: 3
Gate counts: {'cx': 5, 'h': 2, 't': 2, 'tdg': 1}
Operator Norm Distance: 4.998217e-16
Process Fidelity: 1.000000

Circuit:
global phase: 0
                           ┌───┐┌───┐     ┌───┐     
q_0: ───────■───────────■──┤ T ├┤ H ├──■──┤ X ├──■──
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐├───┤└───┘┌─┴─┐└─┬─┘┌─┴─┐
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─────┤ X ├──■──┤ X ├
     └───┘└───┘└─────┘└───┘└───┘     └───┘     └───┘



---
## Unitary 9

In [ ]:
# Unitary 9
qc9, U9 = optimize_and_display("Unitary 9", "unitary9.npy", verbose=False)

Unitary 9

TOTAL T-GATES: 0
Gate counts: {'cx': 2, 's': 1}
Operator Norm Distance: 1.169421e+00
Process Fidelity: 0.625000

Circuit:
global phase: 0.32175
          ┌───┐     
q_0: ──■──┤ X ├─────
     ┌─┴─┐└─┬─┘┌───┐
q_1: ┤ X ├──■──┤ S ├
     └───┘     └───┘



---
## Unitary 10

In [ ]:
# Unitary 10
qc10, U10 = optimize_and_display("Unitary 10", "unitary10.npy", verbose=False)

Unitary 10

TOTAL T-GATES: 0
Gate counts: {'s': 4, 'h': 2}
Operator Norm Distance: 1.089767e+00
Process Fidelity: 0.566066

Circuit:
global phase: 3.8768
     ┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ S ├┤ S ├┤ H ├
     ├───┤├───┤└───┘└───┘
q_1: ┤ S ├┤ S ├──────────
     └───┘└───┘          



---
## Unitary 11

In [ ]:
# Unitary 11
qc11, U11 = optimize_and_display("Unitary 11", "unitary11.npy", verbose=False)

Unitary 11

TOTAL T-GATES: 4
Gate counts: {'cx': 10, 'tdg': 4}
Operator Norm Distance: 3.550831e-16
Process Fidelity: 1.000000

Circuit:
global phase: 2π
                                                                                   
q_0: ──■────────────────■─────────■────────────────────────────────────────■───────
       │                │         │                                        │       
q_1: ──┼────■───────────┼────■────┼────■───────────────────────■───────────┼───────
     ┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐  │    │                       │           │       
q_2: ┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ X ├──┼────┼───────────■───────────┼───────────┼────■──
     └───┘└───┘└─────┘└───┘└───┘┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐
q_3: ───────────────────────────┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ X ├
                                └───┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└───┘



---
## Summary

All 11 unitaries optimized. Each cell displays:
- **TOTAL T-GATES**: Number of T and Tdg gates
- **Gate counts**: Full breakdown of all gates
- **Operator Norm Distance**: How close the circuit is to the target unitary
- **Process Fidelity**: Quantum channel fidelity between circuit and target

---
## Brute Force Analyzer (Integrated)

The brute force analyzer is now integrated into `optimize_unitary.py`. Use these functions to search for optimal low-T circuits for any unitary.

### Available Functions:
- `analyze_unitary(U, name)` - Analyze structure, eigenvalues, and detect patterns
- `brute_force_search(U, max_t, max_cx, timeout)` - Search for optimal circuit
- `quick_search(U, max_t)` - Fast search with limited parameters (60s timeout)
- `compare_circuits(U, circuits_dict)` - Compare multiple circuits
- `full_analysis(U, name, max_t, max_cx)` - Analysis + search combined

### Usage:
```python
# Analyze any unitary (U1-U11)
analyze_unitary(U7, name="Unitary 7")

# Search for optimal circuit
best_qc, best_fid = brute_force_search(U7, max_t=3, max_cx=3)
```

In [ ]:
# Example: Analyze and search for a better circuit for any unitary
# Just change the variable name to analyze a different one (U1-U11)

# Analyze the unitary structure
analyze_unitary(U7, name="Unitary 7")

In [ ]:
# Brute force search for optimal circuit (adjust max_t and max_cx as needed)
# Warning: Higher values take longer to search!

best_qc, best_fid = brute_force_search(U7, max_t=3, max_cx=3, verbose=True)

In [ ]:
# Compare your optimized circuit against the best found by brute force
if best_qc is not None:
    compare_circuits(U7, {
        "Optimized (from cell)": qc7,
        "Brute Force Best": best_qc
    })